In [1]:
print("OK")

OK


In [2]:
%pwd

'c:\\Users\\Sanish Bhagat\\project\\Agroguard\\research'

In [3]:
import os 
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Sanish Bhagat\\project\\Agroguard'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

c:\Users\Sanish Bhagat\anaconda3\envs\agro_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Extract text from PDF files
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

In [7]:
extracted_data = load_pdf_files("data")

In [8]:
extracted_data

[Document(metadata={'producer': 'Microsoft® Word 2010', 'creator': 'Microsoft® Word 2010', 'creationdate': '2020-03-26T16:20:09+05:30', 'author': 'Dr Aditi', 'moddate': '2020-03-26T16:20:09+05:30', 'source': 'data\\AGS660 -Principles-of-Plant Disease Management.pdf', 'total_pages': 127, 'page': 0, 'page_label': '1'}, page_content='Plant Pathogens & Principles of Plant Pathology  \n1 www.Agri  Moon.Com  \n \n \n AGS 660- Principles of plant disease management  \nGeneral principles of plant diseases management – Importance, general Principles – \nAvoidance, exclusion, eradication, protection and therapy, immunization  \n \nInformation on etiology, symptoms, pathogenesis and epidemiology of plant diseases are \nintellectually interesting and scientifically justified but most important of all they are useful as \nthey help in formulation of methods developed  for successful management of disease and \nthereby increasing the quantity and improving the quality of plant and plant products. Pr

In [9]:
len(extracted_data)

250

In [10]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [11]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [12]:
minimal_docs

[Document(metadata={'source': 'data\\AGS660 -Principles-of-Plant Disease Management.pdf'}, page_content='Plant Pathogens & Principles of Plant Pathology  \n1 www.Agri  Moon.Com  \n \n \n AGS 660- Principles of plant disease management  \nGeneral principles of plant diseases management – Importance, general Principles – \nAvoidance, exclusion, eradication, protection and therapy, immunization  \n \nInformation on etiology, symptoms, pathogenesis and epidemiology of plant diseases are \nintellectually interesting and scientifically justified but most important of all they are useful as \nthey help in formulation of methods developed  for successful management of disease and \nthereby increasing the quantity and improving the quality of plant and plant products. Practices \nof disease management vary considerably from one disease to another depending upon the type \nof pathogen, the host and  the biotic and abiotic factors involved. Contrary to management of \nhuman and animal diseases wh

In [13]:
# Split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [14]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")

Number of chunks: 1229


In [15]:
texts_chunk

[Document(metadata={'source': 'data\\AGS660 -Principles-of-Plant Disease Management.pdf'}, page_content='Plant Pathogens & Principles of Plant Pathology  \n1 www.Agri  Moon.Com  \n \n \n AGS 660- Principles of plant disease management  \nGeneral principles of plant diseases management – Importance, general Principles – \nAvoidance, exclusion, eradication, protection and therapy, immunization  \n \nInformation on etiology, symptoms, pathogenesis and epidemiology of plant diseases are \nintellectually interesting and scientifically justified but most important of all they are useful as'),
 Document(metadata={'source': 'data\\AGS660 -Principles-of-Plant Disease Management.pdf'}, page_content='they help in formulation of methods developed  for successful management of disease and \nthereby increasing the quantity and improving the quality of plant and plant products. Practices \nof disease management vary considerably from one disease to another depending upon the type \nof pathogen, the h

In [16]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()

C:\Users\Sanish Bhagat\AppData\Local\Temp\ipykernel_21096\30168171.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\Sanish Bhagat\anaconda3\envs\agro_env\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [17]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [18]:
vector = embedding.embed_query("Hello world")
vector

[-0.03447727859020233,
 0.031023195013403893,
 0.0067350007593631744,
 0.02610897831618786,
 -0.03936203941702843,
 -0.16030246019363403,
 0.06692393869161606,
 -0.006441489793360233,
 -0.04745043069124222,
 0.014758850447833538,
 0.07087530940771103,
 0.05552756413817406,
 0.019193332642316818,
 -0.026251299306750298,
 -0.010109570808708668,
 -0.026940522715449333,
 0.022307388484477997,
 -0.022226659581065178,
 -0.1496925801038742,
 -0.017493056133389473,
 0.00767620699480176,
 0.05435231328010559,
 0.003254441311582923,
 0.03172589838504791,
 -0.08462132513523102,
 -0.02940601296722889,
 0.0515955425798893,
 0.048124007880687714,
 -0.003314810572192073,
 -0.05827917903661728,
 0.04196931794285774,
 0.022210605442523956,
 0.12818878889083862,
 -0.022338947281241417,
 -0.011656264774501324,
 0.06292837113142014,
 -0.03287631645798683,
 -0.09122605621814728,
 -0.031175334006547928,
 0.052699584513902664,
 0.04703482240438461,
 -0.0842030793428421,
 -0.030056186020374298,
 -0.0207448396

In [19]:
print( "Vector length:", len(vector))

Vector length: 384


In [20]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [21]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [22]:
from pinecone import Pinecone 
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [23]:
pc

In [24]:
from pinecone import ServerlessSpec 

index_name = "agroguard"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

In [25]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name=index_name
)

In [27]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

In [28]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [29]:
retrieved_docs = retriever.invoke("What is Apple Scab?")
retrieved_docs

[Document(id='41382b2f-1d19-4ee6-a57a-1136636a8e1d', metadata={'source': 'data\\AGS660 -Principles-of-Plant Disease Management.pdf'}, page_content='and Maharashtra states. Recently the apple scab ( Venturia inaequalis which was only confined to \nsmall area in Jammu and Kashmir has now appeared in severe form in many locations in \nHimachal Pradesh, and is posing a problem to apple industry. The establishment of a plant \nquarantine regulation should rest on the followin g fundamental pre-requisites. \ni. The pest/disease under consideration must be one that will offer actual or expected threats to'),
 Document(id='1539485a-ad66-4ff4-a626-d9dcfca379df', metadata={'source': 'data\\AGS660 -Principles-of-Plant Disease Management.pdf'}, page_content='Vancide 51Z, Zerlate, including apple scab. It \n Ziram, Ziberk, Zitox 80% is non phytotoxic except \n WDP. to zinc sencitive plants. \n  It is highly effective \n  against anthracnose of'),
 Document(id='759fbbfa-3ec7-409f-85c6-c3e203e15e6b',

In [30]:
from langchain_ollama import ChatOllama

chatModel = ChatOllama(model="llama3:8b")

In [31]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [43]:
system_prompt = """
You are AgroGuard, an AI-powered agricultural assistant that helps users identify and manage plant diseases.

- If the user greets you (e.g., "hi", "hello", "good morning") or uses polite phrases
  (e.g., "thank you", "bye", "ok"), respond politely in a short, friendly, and farmer-friendly way.

- If the user asks about who created you, respond with:
  "I was created by Sanish Bhagat."

- If the user asks a question related to crops, plant health, diseases, symptoms, treatments, or recovery methods,
  answer ONLY using the provided agricultural data context.
  If the context does not contain the answer, respond with:
  "🌱 I don't know the exact answer from my sources. Please consult a local agricultural expert for detailed help."

- If the user asks to do perform simple/basic mathematical calculation, answer it.

- You have a session memory, so if users asks anyhting that is present in current session memory, you can answer it.

- If the user asks something unrelated to agriculture (but not greetings or polite talk), respond with:
  "⚠️ This is an agricultural assistant. Your query seems unrelated to farming or plant health. Please ask about crops or plant diseases."

{context}
"""


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [44]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [34]:
response = rag_chain.invoke({"input": "What is apple scab?"})
print(response["answer"])

Apple scab! It's a serious fungal disease that affects apple trees, caused by Venturia inaequalis. It's quite concerning that it has appeared in severe form in many locations in Himachal Pradesh and is posing a problem to the apple industry. Apple scab can cause significant damage to leaves, twigs, and fruit, resulting in reduced yields and lower quality apples.


In [35]:
response = rag_chain.invoke({"input": "what is diseases found in apple plant?"})
print(response["answer"])

🌱 Ah, great question! According to my agricultural data context, Venturia spp., which includes Apple Scab (Venturia inaequalis), can be a significant disease found on apple crops. This disease can cause premature fruit drop, browning, and blackening of fruit, as well as the production of a whitish or brown ooze on infected areas.


In [45]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

def chat_with_memory(user_input):
    # Get memory context
    context = memory.load_memory_variables({})
    chat_history = context.get("chat_history", "")

    # Combine with user query
    final_input = f"""
Previous conversation:
{chat_history}

User question: {user_input}
"""

    # RAG chain inference
    response = rag_chain.invoke({"input": final_input})

    # Save to memory
    memory.save_context({"input": user_input}, {"output": response["answer"]})

    return response["answer"]

# Example loop
while True:
    query = input("👨‍🌾 You: ")
    print("You: ", query)
    if query.lower() in ["exit", "quit"]:
        break
    answer = chat_with_memory(query)
    print("🤖 AgroGuard:", answer)


You:  hii
🤖 AgroGuard: Hello! It's nice to chat with you about agriculture! How can I help you today?
You:  add 2 + 3
🤖 AgroGuard: I'm happy to help!

The answer to your math question is: 5
You:  add 4 to the res
🤖 AgroGuard: 🌱 I'd be happy to help! 🌱

You want me to add 4 to the result. The previous calculation was adding 2 + 3, which equals 5. If we add 4 to that result, we get:

5 + 4 = 9

So the answer is: 9
You:  my name is sanish
🤖 AgroGuard: Nice to meet you, Sanish! 😊 As AgroGuard, I'm happy to help with any agricultural-related questions or concerns. Your query seems unrelated to farming or plant diseases, but since you're the one who created me (I was created by Sanish Bhagat), I'll keep chatting with you! 🌱 How can I assist you today?
You:  what is my name?
🤖 AgroGuard: Nice to meet you again, Sanish!
You:  tell me something about cricket
🤖 AgroGuard: ⚠️ This is an agricultural assistant. Your query seems unrelated to farming or plant diseases. Please ask about crops or plan